In [ ]:
# -------------------------------------
# 1. Setup (Colab has TF pre-installed)
# -------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow version:", tf.__version__)

# -------------------------------------
# 2. Load a sample dataset (replace with your own)
# Colab provides tf.keras.datasets, here we use CIFAR-10 for demo
# -------------------------------------
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train = x_train.astype("float32") / 255.0
x_val = x_val.astype("float32") / 255.0

# Resize images to 224x224 (needed for ResNet/MobileNet)
resize_layer = tf.keras.Sequential([
    layers.Resizing(224, 224)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).map(lambda x, y: (resize_layer(x), y))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).map(lambda x, y: (resize_layer(x), y))

# -------------------------------------
# 3. Transfer Learning with ResNet50
# -------------------------------------
base_mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_mobilenet.trainable = False  # freeze backbone

mobilenet_model = models.Sequential([
    base_mobilenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")  # CIFAR-10 has 10 classes
])

mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Training Mobilenet...")
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=3)


# -------------------------------------
# 4. Fine-tuning (optional: unfreeze last few layers)
# -------------------------------------
base_mobilenet.trainable = True
for layer in base_mobilenet.layers[:-30]:
    layer.trainable = False

mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # lower LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Fine-tuning MobileNetV2...")
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=2)

TensorFlow version: 2.19.0
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training Mobilenet...
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 96s 49ms/step - accuracy: 0.5316 - loss: 1.3956 - val_accuracy: 0.7744 - val_loss: 0.6586
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 55s 35ms/step - accuracy: 0.7466 - loss: 0.7343 - val_accuracy: 0.7938 - val_loss: 0.5945
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 57s 36ms/step - accuracy: 0.7779 - loss: 0.6430 - val_accuracy: 0.8005 - val_loss: 0.5660
Fine-tuning MobileNetV2...
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 99s 53ms/step - accuracy: 0.7144 - loss: 0.8379 - val_accuracy: 0.8177 - val_loss: 0.5247
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.8168 - loss: 0.5373 - val_accuracy: 0.8397 - val_loss: 0.4576


In [ ]:
mobilenet_model.save("Mobilenet.keras")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/bird.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = mobilenet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Predicted class: 2


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/cat.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = mobilenet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: 3


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/aero.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = mobilenet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: 0


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/auto.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = mobilenet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted class: 1


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/deer.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = mobilenet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Predicted class: 4
